# **Dependancies**

In [358]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import *
from keras.callbacks import *
import os
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from commons import mean_absolute_percentage_error
from keras.layers import *
from sklearn.pipeline import Pipeline
from keras.utils import to_categorical
from tensorflow.keras.models import load_model
import tensorflow as tf

# **Loading Data**

In [359]:
PATH_TO_DATA = "cls_interval121-24.csv"
data = pd.read_csv(PATH_TO_DATA)

In [360]:
data.shape

(1097, 26)

In [361]:
X = data.iloc[:,:-1]

In [362]:
y=data.iloc[:,-1:]

In [363]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.2, train_size=0.8, shuffle=False, random_state=7)

In [364]:
X_train.shape

(877, 25)

In [365]:
y_train

,category
0,1
1,0
2,0
3,1
4,1
...,...
872,0
873,1
874,1
875,0


In [366]:
estimators=[]

In [367]:
estimators.append(['robust',RobustScaler()])

In [368]:
estimators.append(['mixmax',MinMaxScaler()])

In [369]:
scale=Pipeline(estimators,verbose=True)

In [370]:
X_train=scale.fit_transform(X_train)

[Pipeline] ............ (step 1 of 2) Processing robust, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing mixmax, total=   0.0s


In [371]:
X_test=scale.transform(X_test)

In [372]:
X_train=np.reshape(X_train,(X_train.shape[0],1,X_train.shape[1]))

In [373]:
X_test=np.reshape(X_test,(X_test.shape[0],1,X_test.shape[1]))

In [374]:
y_train=y_train.values

In [375]:
y_train

array([[1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
    

In [376]:
unique1, id1 = np.unique(y_train, return_inverse=True)

In [377]:
y_train=to_categorical(id1,num_classes=2)

In [378]:
y_train

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [379]:
X_train.shape

(877, 1, 25)

In [380]:
y_train.shape

(877, 2)

In [381]:
y_train

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [382]:
y_train=np.reshape(y_train, (y_train.shape[0],1,y_train.shape[1]))

In [383]:
y_test=y_test.values

In [384]:
#y_test

In [385]:
#unique1, id1 = np.unique(y_test, return_inverse=True)
#y_test = to_categorical(id1, num_classes=2)

In [386]:
#y_test=np.reshape(y_test, (y_test.shape[0],1,y_test.shape[1]))

In [387]:
y_test

array([[1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
    

# **Model Architecture + Training**

In [388]:

def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [389]:
adam = optimizers.Adam(learning_rate=lr_schedule(0), amsgrad=True)


Learning rate:  0.001


In [390]:
model = Sequential()
model.add(Bidirectional(LSTM(80, return_sequences=True, activation='relu'), input_shape=(1, X_train.shape[2])))
model.add(Bidirectional(LSTM(100, return_sequences=True, activation='relu')))
#model.add(Bidirectional(LSTM(40, return_sequences=True, activation='relu')))
model.add(Dense(2,activation='softmax'))
model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=['accuracy'])

E:\DS440proj\BTCpred\.venv\lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [391]:
mcp_save = ModelCheckpoint('LSTM_cls_interval3.keras', save_best_only=True, monitor='val_loss', mode='max')
earlyStopping = EarlyStopping(monitor='val_loss', patience=50, verbose=1, mode='max')

In [392]:
model.fit(X_train,y_train, epochs=1500, batch_size=32, validation_split=0.1,validation_freq=1, shuffle=False, callbacks=[mcp_save,earlyStopping])

Epoch 1/1500
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.4491 - loss: 0.6934 - val_accuracy: 0.4773 - val_loss: 0.6939
Epoch 2/1500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5094 - loss: 0.6926 - val_accuracy: 0.4773 - val_loss: 0.6939
Epoch 3/1500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5353 - loss: 0.6919 - val_accuracy: 0.4659 - val_loss: 0.6939
Epoch 4/1500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5439 - loss: 0.6909 - val_accuracy: 0.4659 - val_loss: 0.6937
Epoch 5/1500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5494 - loss: 0.6895 - val_accuracy: 0.5000 - val_loss: 0.6938
Epoch 6/1500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5603 - loss: 0.6878 - val_accuracy: 0.5227 - val_loss: 0.6935
Epoch 7/1500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5593 - loss: 0.6860 - val_accuracy: 0.5341 - val_loss: 0.6939
Epoch 8/1500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5598 - loss: 0.6845 - val_accuracy: 0

# **Testing Model**

In [393]:
#prediction_model = load_model('trained_models/LSTM_cls_interval3.keras',compile=False)

In [394]:
# Make predictions with the model
predictions = model.predict(X_test)
# Using np.argmax on the last axis to convert probability predictions to class index predictions
y_pred = np.argmax(predictions, axis=-1)  # This will keep the first two dimensions intact

# Check if y_pred shape matches y_test's structure without the class probabilities
print("Shape of y_pred:", y_pred.shape)



7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Shape of y_pred: (220, 1)


In [395]:
predictions

array([[[1.00000000e+00, 0.00000000e+00]],

       [[1.00000000e+00, 0.00000000e+00]],

       [[1.00000000e+00, 0.00000000e+00]],

       [[1.00000000e+00, 0.00000000e+00]],

       [[1.00000000e+00, 0.00000000e+00]],

       [[1.00000000e+00, 0.00000000e+00]],

       [[1.00000000e+00, 0.00000000e+00]],

       [[1.00000000e+00, 0.00000000e+00]],

       [[1.00000000e+00, 0.00000000e+00]],

       [[1.00000000e+00, 0.00000000e+00]],

       [[1.00000000e+00, 0.00000000e+00]],

       [[1.00000000e+00, 0.00000000e+00]],

       [[4.86291792e-05, 9.99951363e-01]],

       [[4.79976507e-03, 9.95200157e-01]],

       [[9.61177078e-24, 1.00000000e+00]],

       [[3.13437157e-24, 1.00000000e+00]],

       [[4.99760790e-04, 9.99500275e-01]],

       [[4.01269421e-02, 9.59873080e-01]],

       [[9.53072846e-01, 4.69271205e-02]],

       [[9.61295724e-01, 3.87042128e-02]],

       [[1.27517101e-30, 1.00000000e+00]],

       [[9.99771774e-01, 2.28186182e-04]],

       [[9.99405026e-01, 5.94944

In [396]:
y_pred

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
    

In [397]:
y_test.shape

(220, 1)

In [398]:
acc=accuracy_score(y_test,y_pred)
acc

0.4818181818181818

In [399]:
f1=f1_score(y_test,y_pred,average='binary')
f1

0.5086206896551724

In [400]:
auc=roc_auc_score(y_test,y_pred)
auc

0.4949891067538127

In [402]:
y_prob=[model.predict(X_test).max() for i in range(len(y_test))]

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 975us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 917us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━

In [403]:
print(classification_report(y_test,y_pred,labels=[0,1], target_names=['decrease','increase']))

              precision    recall  f1-score   support

    decrease       0.38      0.55      0.45        85
    increase       0.61      0.44      0.51       135

    accuracy                           0.48       220
   macro avg       0.50      0.49      0.48       220
weighted avg       0.52      0.48      0.49       220



In [404]:
predictions=pd.DataFrame(zip(np.ravel(y_test),np.ravel(y_pred)),columns=['y_test','y_pred'])

In [405]:
predictions

,y_test,y_pred
0,1,0
1,0,0
2,1,0
3,1,0
4,0,0
...,...,...
215,0,0
216,0,0
217,1,0
218,0,0
